In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader


loader=UnstructuredURLLoader( urls=[
        "https://www.moneycontrol.com/news/business/ipo/icici-prudential-amc-kicks-off-mega-ipo-with-record-syndicate-filing-likely-by-june-end-with-launch-aimed-for-q3-13108573.html",
        "https://www.moneycontrol.com/news/business/markets/trade-setup-for-june-11-top-15-things-to-know-before-the-opening-bell-13109153.html"
    ],
   
)

documents=loader.load()


In [16]:
documents[0].page_content

'English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹50 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nNetwork 18\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_BUSINESS_AS/MC_ENG_ROS_NWS_BUS_AS_ATF_728\n\nMoneycontrol\n\nGo PRO NowPRO\n\nMoneycontrol PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nSubscription Products\n\nprofile\n\nAmbareesh Baliga\n\nprofile\n\nCK Narayan\n\nprofile\n\nT Gnanasekar\n\nprofile\n\nPower Your Trade\n\nprofile\n\nCurrencies with Mecklai Financial\n\nprofile\n\nStock Reports\n\nHomeNewsBusinessIPOICICI Prudent

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)
splits = text_splitter.split_documents(documents)

In [25]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# Embed with HuggingFace
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [28]:
from langchain.vectorstores import Chroma
# Store vectors in Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)


In [31]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

if not groq_api_key:
    raise ValueError("GROQ_API_KEY is not set in your .env file.")
else:
    print("GroQ API key loaded successfully.")
    os.environ["GROQ_API_KEY"] = groq_api_key

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq", temperature=0.7, api_key=groq_api_key)


GroQ API key loaded successfully.


In [52]:
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
query = "What is the latest news on ICICI Prudential AMC's IPO?"
result = qa({"query": query})
print("Answer:", result['result'])
print("Source Documents:", [doc.metadata['source'] for doc in result['source_documents']])

Answer: According to the latest news, ICICI Prudential AMC's IPO will be a pure Offer For Sale (OFS) by Prudential, with no participation by ICICI Bank. The filing is likely to happen by the end of June, and the launch is aimed for the third quarter.
Source Documents: ['https://www.moneycontrol.com/news/business/ipo/icici-prudential-amc-kicks-off-mega-ipo-with-record-syndicate-filing-likely-by-june-end-with-launch-aimed-for-q3-13108573.html', 'https://www.moneycontrol.com/news/business/ipo/icici-prudential-amc-kicks-off-mega-ipo-with-record-syndicate-filing-likely-by-june-end-with-launch-aimed-for-q3-13108573.html']
